# Homework 7

## CSCI E-82A


In the a previous homework assignment, you used two different dynamic programming algorithms to solve a robot navigation problem by finding optimal paths to a goal in a simplified warehouse environment. Now you will use first visit Monte Carlo reinforcement learning to find optimal paths in the same environment.

The configuration of the warehouse environment is illustrated in the figure below.

<img src="GridWorldFactory.JPG" alt="Drawing" style="width:200px; height:200px"/>
<center> **Grid World for Factory Navigation Example** </center>

The goal is for the robot to deliver some material to position (state) 12, shown in blue. Since there is a goal state or **terminal state** this an **episodic task**. 

There are some barriers comprised of the states $\{ 6, 7, 8 \}$ and $\{ 16, 17, 18 \}$, shown with hash marks. In a real warehouse, these positions might be occupied by shelving or equipment. We do not want the robot to hit these barriers. Thus, we say that transitioning to these barrier states is **taboo**.

As before, we do not want the robot to hit the edges of the grid world, which represent the outer walls of the warehouse. 

## Representation

As with many such problems, the starting place is creating the **representation**. In the cell below encode your representation for the possible action-state transitions. From each state there are 4 possible actions:
- up, u
- down, d,
- left, l
- right, r

There are a few special cases you need to consider:
- Any action transitioning state off the grid or into a barrier should keep the state unchanged. 
- Any action in the goal state keeps the state unchanged. 
- Any transition within the taboo (barrier) states can keep the state unchanged. If you experiment, you will see that other encodings work as well since the value of a barrier states are always zero and there are no actions transitioning into these states. 

> **Hint:** It may help you create a pencil and paper sketch of the transitions, rewards, and probabilities or policy. This can help you to keep the bookkeeping correct. 

In [1]:
## import numpy for latter
import numpy as np
import numpy.random as nr



In [2]:
neighbors = {0:{'u':0, 'd':5, 'l':0, 'r':1},
          1:{'u':1, 'd':1, 'l':0, 'r':2},
          2:{'u':2, 'd':2, 'l':1, 'r':3},
          3:{'u':3, 'd':3, 'l':2, 'r':4},
          4:{'u':4, 'd':9, 'l':3, 'r':4},
          
          5:{'u':0, 'd':10, 'l':5, 'r':5},
          6:{'u':6, 'd':6, 'l':6, 'r':6},###barrier
          7:{'u':7, 'd':7, 'l':7, 'r':7},###barrier
          8:{'u':8, 'd':8, 'l':8, 'r':8},###barrier
          9:{'u':4, 'd':14, 'l':9, 'r':9},
          
          10:{'u':5, 'd':15, 'l':10, 'r':11},
          11:{'u':11, 'd':11, 'l':10, 'r':12},
          12:{'u':12, 'd':12, 'l':12, 'r':12},#goal
          13:{'u':13, 'd':13, 'l':12, 'r':14},
          14:{'u':9, 'd':19, 'l':13, 'r':14},
          
          15:{'u':10, 'd':20, 'l':15, 'r':15},
          16:{'u':16, 'd':16, 'l':16, 'r':16},###barrier
          17:{'u':17, 'd':17, 'l':17, 'r':17},###barrier
          18:{'u':18, 'd':18, 'l':18, 'r':18},###barrier
          19:{'u':14, 'd':24, 'l':19, 'r':19},
          
          20:{'u':15, 'd':20, 'l':20, 'r':21},
          21:{'u':21, 'd':21, 'l':20, 'r':22},
          22:{'u':22, 'd':22, 'l':21, 'r':23},
          23:{'u':23, 'd':23, 'l':22, 'r':24},
          24:{'u':19, 'd':24, 'l':23, 'r':24}}

You need to define the initial transition probabilities for the Markov process. Set the probabilities for each transition as a **uniform distribution** leading to random action by the robot. 

> **Note:** As these are just starting values, the exact values of the transition probabilities are not actually all that important in terms of solving the MC RL problem. Also, notice that it does not matter how the taboo state transitions are encoded. The point of the MC RL algorithm is to learn a model including the transition policy.

In [3]:
policy = {0:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
          1:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
          2:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
          3:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
          4:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
          
          5:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
          6:{'u':0, 'd':0, 'l':0, 'r':0},###barrier
          7:{'u':0, 'd':0, 'l':0, 'r':0},###barrier
          8:{'u':0, 'd':0, 'l':0, 'r':0},###barrier
          9:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
          
          10:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
          11:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
          12:{'u':0, 'd':0, 'l':0, 'r':0},#goal
          13:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
          14:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
          
          15:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
          16:{'u':0, 'd':0, 'l':0, 'r':0},###barrier
          17:{'u':0, 'd':0, 'l':0, 'r':0},###barrier
          18:{'u':0, 'd':0, 'l':0, 'r':0},###barrier
          19:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
          
          20:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
          21:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
          22:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
          23:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
          24:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25}}

The robot receives the following rewards:
- 10 for entering position 0. 
- -1 for attempting to leave the grid. In other words, we penalize the robot for hitting the edges of the grid.  
- -0.1 for all other state transitions, which is the cost for the robot to move from one state to another. If we did not have this penalty, the robot could follow any random plan to the goal which did not hit the edges. 

This **reward structure is unknown to the MC RL agent**. The agent must **learn** the rewards by sampling the environment. 

In the code cell below encode your representation of this reward structure you will use in your simulated environment.  

In [4]:
reward = {0:{'u':-1, 'd':-0.1, 'l':-1, 'r':-0.1},
          1:{'u':-1, 'd':-1, 'l':-0.1, 'r':-0.1},
          2:{'u':-1, 'd':-1, 'l':-0.1, 'r':-0.1},
          3:{'u':-1, 'd':-1, 'l':-0.1, 'r':-0.1},
          4:{'u':-1, 'd':-0.1, 'l':-1, 'r':-0.1},
          
          5:{'u':-0.1, 'd':-0.1, 'l':-1, 'r':-1},
          6:{'u':0, 'd':0, 'l':0, 'r':0},###barrier
          7:{'u':0, 'd':0, 'l':0, 'r':0},###barrier
          8:{'u':0, 'd':0, 'l':0, 'r':0},###barrier
          9:{'u':-0.1, 'd':-0.1, 'l':-1, 'r':-1},
          
          10:{'u':-0.1, 'd':-0.1, 'l':-1, 'r':-0.1},
          11:{'u':-1, 'd':-1, 'l':-0.1, 'r':10},
          12:{'u':0, 'd':0, 'l':0, 'r':0},#goal
          13:{'u':-1, 'd':-1, 'l':10, 'r':-0.1},
          14:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-1},
          
          15:{'u':-0.1, 'd':-0.1, 'l':-1, 'r':-1},
          16:{'u':0, 'd':0, 'l':0, 'r':0},###barrier
          17:{'u':0, 'd':0, 'l':0, 'r':0},###barrier
          18:{'u':0, 'd':0, 'l':0, 'r':0},###barrier
          19:{'u':-0.1, 'd':-0.1, 'l':-1, 'r':-1},
          
          20:{'u':-0.1, 'd':-1, 'l':-0.1, 'r':-0.1},
          21:{'u':-1, 'd':-1, 'l':-0.1, 'r':-0.1},
          22:{'u':-1, 'd':-1, 'l':-0.1, 'r':-0.1},
          23:{'u':-1, 'd':-1, 'l':-0.1, 'r':-0.1},
          24:{'u':-0.1, 'd':-1, 'l':-0.1, 'r':-1}}

You will find it useful to create a list of taboo states, which you can encode in the cell below.

In [5]:
taboo = [6,7,8,16,17,18]
taboo+[12]

[6, 7, 8, 16, 17, 18, 12]

## Policy Evaluation

With your representations defined, you can now create and test functions to perform MC RL **policy evaluation**. You will need these functions for your policy improvement code. 

As a first step you will need a function to generate episodes given the policy in the warehouse environment. You are welcome to start with the `MC_generate_episode` function from the MC RL notebook. However, keep in mind that you must modify this code to correctly treat the taboo states of the barrier. Specifically, taboo states will not be visited on the random walk. 

Set a seed of `4567` and execute your code to test it.  

In [6]:
def MC_generate_episode(policy, neighbors, terminal):
    ## List of states which might be visited in episode
    n_states = len(policy)
#    visited_state = [0] * n_states
    states = list(neighbors.keys())
    
    ## Random starting state for this episode, but can't be the terminal state
    current_state = nr.choice(states, size = 1)[0]
    while(current_state in taboo+[terminal]): # Keep trying to not use terminal state to start
        current_state = nr.choice(states, size = 1)[0]
            
    ## Take a random walk trough the states until we get to the terminal state
    ## We do some bookkeeping to ensure we only visit states once.
    visited = [] # List of states visited on random walk
    while(current_state != terminal): # Stop when at terminal state
        ## Probability of state transition given policy
        probs = list(policy[current_state].values())
        ## Find next state to transition to
        next_state = nr.choice(list(neighbors[current_state].values()), size = 1, p = probs)[0]
        visited.append(next_state)
        current_state = next_state  
    return(visited)    
    


In [7]:
nr.seed(4567)    

for i in MC_generate_episode(policy, neighbors, 12):
    print(i)
    print(i in taboo)


20
False
20
False
20
False
21
False
21
False
21
False
21
False
21
False
22
False
23
False
23
False
24
False
24
False
23
False
23
False
23
False
22
False
21
False
21
False
22
False
21
False
20
False
21
False
21
False
21
False
20
False
20
False
15
False
10
False
15
False
15
False
20
False
21
False
22
False
22
False
22
False
23
False
23
False
24
False
23
False
22
False
22
False
21
False
20
False
21
False
22
False
23
False
24
False
24
False
19
False
19
False
19
False
19
False
14
False
14
False
14
False
13
False
13
False
14
False
9
False
9
False
9
False
4
False
3
False
2
False
3
False
4
False
4
False
9
False
9
False
9
False
9
False
9
False
14
False
14
False
19
False
19
False
24
False
19
False
24
False
23
False
23
False
24
False
23
False
24
False
23
False
23
False
23
False
22
False
22
False
23
False
22
False
22
False
22
False
22
False
22
False
23
False
24
False
24
False
23
False
23
False
22
False
23
False
22
False
22
False
21
False
22
False
21
False
20
False
20
False
15
False
15
False
20
Fal

Examine your results and answer the following questions to ensure your code has operated correctly:
1. Have any taboo states been visited? ANS: No. 
2. Does the random walk end at the terminal state? ANS: Yes, at 12.

Next, you need to create a function to compute the first visit action values along the random walk path. You are welcome to start with the `MC_state_values` function from the MC RL notebook. Notice that this function is incorrectly named, as it actually does compute action values.  

Execute your function for 1,000 episodes. Make sure your code has 0 action values for the taboo states and the goal (terminal) state. If not, something is likely wrong with your random walk generation. 

In [8]:
def MC_state_values(policy, neighbors, rewards, terminal, episodes = 1):
    '''Function for first visit Monte Carlo on GridWorld.'''
    ## Create list of states 
    states = list(policy.keys())
    n_states = len(states)
    
    ## An array to hold the accumulated returns as we visit states
    G = np.zeros((episodes,n_states))
    
    ## An array to keep track of how many times we visit each state so we can 
    ## compute the mean
    n_visits = np.zeros((n_states))
    
    ## Iterate over the episodes
    for i in range(episodes):
        ## For each episode we use a list to keep track of states we have visited.
        ## Once we visit a state we need to accumulate values to get the returns
        states_visited = []
   
        ## Get a path for this episode
        visit_list = MC_generate_episode(policy, neighbors, terminal)
        current_state = visit_list[0]
        for state in visit_list[0:]: 
            ## list of states we can transition to from current state
            transition_list = list(neighbors[current_state].values())
            
            if(state in transition_list): # Make sure the transistion is allowed
                transition_index = transition_list.index(state)   
  
                ## find the action value for the state transition
                v_s = list(rewards[current_state].values())[transition_index]
   
                ## Mark that the current state has been visited 
                if(state not in states_visited): states_visited.append(current_state)  
                ## Loop over the states already visited to add the value to the return
                for visited in states_visited:
                    G[i,visited] = G[i,visited] + v_s
                    n_visits[visited] = n_visits[visited] + 1.0
            ## Update the current state for next transition
            current_state = state   
    
    ## Compute the average of G over the episodes are return
    n_visits = [nv if nv != 0.0 else 1.0 for nv in n_visits]
    returns = np.divide(np.sum(G, axis = 0), n_visits)   
    return(returns)              


In [9]:
nr.seed(4567)
returns = MC_state_values(policy, neighbors, reward, terminal = 12, episodes = 1000)
np.array(returns).reshape((5,5))

array([[-0.43018543, -0.43923329, -0.44029618, -0.45050552, -0.43947299],
       [-0.4109179 ,  0.        ,  0.        ,  0.        , -0.41474819],
       [-0.35659968,  0.03120994,  0.        ,  0.09269507, -0.36515248],
       [-0.3954133 ,  0.        ,  0.        ,  0.        , -0.403777  ],
       [-0.4127993 , -0.42663166, -0.43609374, -0.43173438, -0.42296307]])

Examine your results and answer the following questions to ensure you action value function operates correctly:
1. Are the values of the taboo states 0? ANS: Yes, you can see above.
2. Are the states with the highest values adjacent to the terminal state? ANS: Yes, 11, 13 values are highest, not same though.
3. Are the values of the states decreasing as the distance from the terminal state increases? ANS: Yes, but not symmetric. Weirdly.


## Policy Improvement

Now that you have your representation and a functions to perform the MC policy evaluation you have everything you need to apply the policy improvement algorithm to create an optimal policy for the robot to reach the goal. 

If your policy evaluation functions work correctly, you should be able to use the `MC_optimal_policy` function from the MC RL notebook. Execute your code using 5 cycles of 1000 episodes for policy improvement. Set $\epsilon = 0.01$. Computation may take some time. Make sure you print the state values as well as the policy you discovered. 

> **Note:** There is some chance the algorithm will not converge from certain starting states. This seems to result from accumulating small errors over the many iterations. The solution should be to change seeds. 

In [10]:
import copy
def MC_optimal_policy(policy, neighbors, rewards, terminal, episodes = 10, cycles = 10, epsilon = 0.05):
    ## Create a working cooy of the initial policy
    current_policy = copy.deepcopy(policy)
    
    ## Loop over a number of cycles of GPI
    for _ in range(cycles):
        ## First compute the average returns for each of the states. 
        ## This is the policy evaluation phase
        returns = MC_state_values(current_policy, neighbors, rewards, terminal = terminal, episodes = episodes)
        
        ## We want max Q for each state, where Q is just the difference 
        ## in the values of the possible state transition
        ## This is the policy evaluation phase
        for s in current_policy.keys(): # iterate over all states
            ## Compute Q for each possible state transistion
            ## Start by creating a list of the adjacent states.
            possible_s_prime = neighbors[s]
            neighbor_states = list(possible_s_prime.values())
            ## Check if terminal state is neighbor, but state is not terminal.
            if(terminal in neighbor_states and s != terminal):
                ## account for the special case adjacent to goal
                neighbor_Q = []
                for s_prime in possible_s_prime.keys(): # Iterate over adjacent states
                    if(neighbors[s][s_prime] == terminal):  
                         neighbor_Q.append(returns[s])
                    else: neighbor_Q.append(0.0) ## Other transisions have 0 value.   
            else: 
                 ## The other case is rather easy. Compute Q for the transistion to each neighbor           
                 neighbor_values = returns[neighbor_states]
                 neighbor_Q = [n_val - returns[s] for n_val in neighbor_values]
                
            ## Find the index for the state transistions with the largest values 
            ## May be more than one. 
            max_index = np.where(np.array(neighbor_Q) == max(neighbor_Q))[0]  
            
            ## Probabilities of transition
            ## Need to allow for further exploration so don't let any 
            ## transition probability be 0.
            ## Some gymnastics are required to ensure that the probabilities 
            ## over the transistions actual add to exactly 1.0
            neighbors_len = float(len(np.array(neighbor_Q)))
            max_len = float(len(max_index))
            diff = round(neighbors_len - max_len,3)
            prob_for_policy = round(1.0/max_len,3)
            adjust = round((epsilon * (diff)), 3)
            prob_for_policy = prob_for_policy - adjust
            if(diff != 0.0):
                remainder = (1.0 - max_len * prob_for_policy)/diff
            else:
                remainder = epsilon
                                                 
            for i, key in enumerate(current_policy[s]): ## Update policy
                if(i in max_index): current_policy[s][key] = prob_for_policy
                else: current_policy[s][key] = remainder          
                   
    return current_policy
 


In [11]:
nr.seed(4567)    
MC_policy = MC_optimal_policy(policy, neighbors, reward, terminal = 12, episodes = 1000, cycles = 5, 
                              epsilon = 0.01)  
MC_policy

{0: {'d': 0.97,
  'l': 0.010000000000000009,
  'r': 0.010000000000000009,
  'u': 0.010000000000000009},
 1: {'d': 0.010000000000000009,
  'l': 0.97,
  'r': 0.010000000000000009,
  'u': 0.010000000000000009},
 2: {'d': 0.010000000000000009,
  'l': 0.97,
  'r': 0.010000000000000009,
  'u': 0.010000000000000009},
 3: {'d': 0.010000000000000009,
  'l': 0.010000000000000009,
  'r': 0.97,
  'u': 0.010000000000000009},
 4: {'d': 0.97,
  'l': 0.010000000000000009,
  'r': 0.010000000000000009,
  'u': 0.010000000000000009},
 5: {'d': 0.97,
  'l': 0.010000000000000009,
  'r': 0.010000000000000009,
  'u': 0.010000000000000009},
 6: {'d': 0.25, 'l': 0.25, 'r': 0.25, 'u': 0.25},
 7: {'d': 0.25, 'l': 0.25, 'r': 0.25, 'u': 0.25},
 8: {'d': 0.25, 'l': 0.25, 'r': 0.25, 'u': 0.25},
 9: {'d': 0.97,
  'l': 0.010000000000000009,
  'r': 0.010000000000000009,
  'u': 0.010000000000000009},
 10: {'d': 0.010000000000000009,
  'l': 0.010000000000000009,
  'r': 0.97,
  'u': 0.010000000000000009},
 11: {'d': 0.0100

Examine your results and answer the following questions:
1. Based on the action values, does it appear that the algorithm is near convergence with only small changes? ANS: Yes, pretty much.
2. Assume the robot enters the warehouse though a door at state (position) 2. As you did for the DP exercise, create an in illustration of the optimal path to the goal based on the policy your algorithm discovered. Replace the figure below with your illustration.

<img src="HW7.JPG" alt="Drawing" style="width:200px; height:200px"/>
<center> **Replace this diagram with one showing your solution** </center>

Finally, test your improved policy by computing the action values. In the cell below, create and execute the code to display the action values for all the states. 

In [12]:
nr.seed(4567)
returns2 = MC_state_values(MC_policy, neighbors, reward, terminal = 12, episodes = 1000)
np.array(returns2).reshape((5,5))

array([[2.0808656 , 1.56596273, 1.38461538, 1.43333333, 1.92451155],
       [2.8382104 , 0.        , 0.        , 0.        , 2.70530973],
       [4.23679031, 8.94736842, 0.        , 8.66159696, 4.15177903],
       [2.83665768, 0.        , 0.        , 0.        , 2.76753022],
       [2.12977444, 1.60025445, 1.38461538, 1.65454545, 2.01470588]])

Examine your results and answer the following questions. 

1. Do your action values represent a significant improvement over the initial random policy? ANS: 

Quiet a lots. Every value becomes a positive, and the highest one is 8.94 from .0092

2. Do the states near the goal have action values near the terminal reward? ANS: Yes, they are 8.9 and 8.6, almost 10. 

